# Генератор отчета по дивидендам зарубежных эмитентов из Tinkoff Broker API

## Автор

Alexander Gerasiov <a@gerasiov.net>

http://github.com/gerasiov/3ndfl

## Лицензия

GPL версии 2 или более поздняя

## Подробнее

Читайте в README.md

Установка модулей в venv:

In [ ]:
# Install tinkoff-investments into venv or user's dir

#!pip install tinkoff-investments
#!pip install --user tinkoff-investments

In [ ]:
from tinkoff.invest import Client, GenerateBrokerReportRequest, GetDividendsForeignIssuerReportRequest, RequestError
from datetime import datetime, timedelta, timezone
from decimal import Decimal
import csv
import sys
from dateutil import rrule
from time import sleep
from grpc import StatusCode

Генерируем readonly токен в недрах веб-интерфейса и вставляем сюда:

In [ ]:
TOKEN='t....'

Год, за который генерируем отчет:

In [ ]:
YEAR=2022

In [ ]:
account = None
with Client(TOKEN) as client:
    for a in client.users.get_accounts().accounts:
        account = a.id  # нужен дальше в запросе, но устроит любой валидный
#         print(a.id, a.name)
        break

In [ ]:
start_date = datetime(YEAR, 1, 1, tzinfo=timezone.utc)
end_date = datetime(YEAR+1, 1, 1, tzinfo=timezone.utc) - timedelta.resolution
    
REPORT_READY_DELAY = 3

In [ ]:
def ratelimit_retry(func):
    def wrap(*args, **kwargs):
        while True:
            try:
#                 print("call", func)
#                 print("args", args)
#                 print("kwargs", kwargs)
#                 print()
                return func(*args, **kwargs)
            except RequestError as e:
                if e.code == StatusCode.RESOURCE_EXHAUSTED:
                    print(f"retry in {e.metadata.ratelimit_reset} seconds...")
                    sleep(e.metadata.ratelimit_reset)
                elif e.code == StatusCode.INVALID_ARGUMENT and 'try again later' in e.metadata.message:
                    print(f"report not ready, retry in {REPORT_READY_DELAY} seconds...")
                    sleep(REPORT_READY_DELAY)
                else:
                    raise
    return wrap

In [ ]:
records = []

with Client(TOKEN) as client:
    result = ratelimit_retry(client.operations.get_dividends_foreign_issuer)(
                generate_div_foreign_issuer_report=GenerateBrokerReportRequest(
                    account_id=account,
                    from_=start_date,
                    to=end_date
                ))
    if result.generate_div_foreign_issuer_report_response is not None:
        sleep(REPORT_READY_DELAY)
        result = ratelimit_retry(client.operations.get_dividends_foreign_issuer)(
                    get_div_foreign_issuer_report=GetDividendsForeignIssuerReportRequest(
                        task_id = result.generate_div_foreign_issuer_report_response.task_id,
                        page = 0
                    ))
    if result.div_foreign_issuer_report is not None and result.div_foreign_issuer_report.pagesCount:
        assert(result.div_foreign_issuer_report.pagesCount == 1)  # I believe this should not happen
        records += result.div_foreign_issuer_report.dividends_foreign_issuer_report

print("Done!")

In [ ]:
def quote_to_decimal(q):
    NANO_PER_1 = 1000000000
    return Decimal(Decimal(q.units) + Decimal(q.nano) / NANO_PER_1).quantize(Decimal('0.01'))

def strip_name(text):
    for s in (
        "_ORD SHS",
        " ORD SHS",
        " REIT",
        " CL A",
        "Американская депозитарная расписка на обыкновенные акции ",
        ", акции обыкновенные",
        "Американские депозитарные расписки на акции ",
        " PREFF SHS",
        'ГДР ',
    ):
        text = text.replace(s, '')
        
    return text.strip()


# write to file according to format, used in nalog.ipynb
# fields = ['date', 'country', 'currency', 'amount', 'tax', 'name']
def write_to_file(output):
    writer = csv.writer(output, delimiter='\t')

    for r in records:
        writer.writerow([
#             r.record_date.strftime("%d.%m.%Y"),
            r.payment_date.strftime("%d.%m.%Y"),
            r.issuer_country,
            'Россия',  # Destination country
            'divident',  # Income type
            r.currency,
#             r.quantity,
#             quote_to_decimal(r.dividend),
            quote_to_decimal(r.dividend_gross),
#             quote_to_decimal(r.tax),  # could be wrong in API, calculate manually below
            quote_to_decimal(r.dividend_gross) - quote_to_decimal(r.dividend_amount),
#             quote_to_decimal(r.dividend_amount),
            strip_name(r.security_name)
        ])

        
write_to_file(sys.stdout)
with open('dividends.tsv', 'w') as f:
    write_to_file(f)